In [1]:
from llama_index import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms import HuggingFaceLLM

In [2]:
documents = SimpleDirectoryReader('/home/chats/data/YuluGyan').load_data()

In [3]:
from llama_index.prompts.prompts import SimpleInputPrompt

In [4]:
system_prompt = "You are a Customer Support Agent for Yulu a Microbility company.Help Cusotmers regarding their queries in polite manner.If a question does not make any sense, or is not factually coherent,explain why instead of answering something not correct. If you do not know the answer to a question, please do not share false information."

In [5]:
query_wrapper_prompt = SimpleInputPrompt("<|User|>{query_str}<|ASSISTANT|>")

In [6]:
query_wrapper_prompt

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['query_str'], kwargs={}, output_parser=None, template='<|User|>{query_str}<|ASSISTANT|>')

In [ ]:
from huggingface_hub import login
login()

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/chats/.cache/huggingface/token
Login successful


In [8]:
import torch

In [9]:
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="sharpbai/Llama-2-7b-chat",
    model_name="sharpbai/Llama-2-7b-chat",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

/home/chats/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/chats/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                        for more details.
                        

In [12]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)


/home/chats/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [13]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=model,
    embed_model=embed_model
)

AttributeError: 'GPT4All' object has no attribute 'metadata'

In [23]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [24]:
query_engine = index.as_query_engine()


In [ ]:
response = query_engine.query("tell me about general rental and use of yulu vehicles")
print(response)

In [14]:
from langchain.llms import GPT4All

In [18]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


model_path = "./models/ggml-gpt4all-j-v1.3-groovy.bin"

callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model = model_path, callbacks=callbacks, verbose=True)


ValidationError: 1 validation error for GPT4All
__root__
  GPT4All.__init__() got an unexpected keyword argument 'device' (type=type_error)

In [2]:
!pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 111.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.8/239.8 KB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 KB 20.8 MB/s eta 0:00:00


In [ ]:
# Import streamlit for app dev
#import streamlit as st

# Import transformer classes for generaiton
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
# Import torch for datatype attributes 
import torch
# Import the prompt wrapper...but for llama index
from llama_index.prompts.prompts import SimpleInputPrompt
# Import the llama index HF Wrapper
from llama_index.llms import HuggingFaceLLM
# Bring in embeddings wrapper
from llama_index.embeddings import LangchainEmbedding
# Bring in HF embeddings - need these to represent document chunks
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# Bring in stuff to change service context
from llama_index import set_global_service_context
from llama_index import ServiceContext
# Import deps to load documents 
from llama_index import VectorStoreIndex, download_loader
from pathlib import Path

# Define variable to hold llama2 weights naming 
name = "sharpbai/Llama-2-7b-chat"
# Set auth token variable from hugging face 
auth_token = ""

def get_tokenizer_model():
    # Create tokenizer
    tokenizer = AutoTokenizer.from_pretrained(name, cache_dir='./model/', use_auth_token=auth_token)

    # Create model
    model = AutoModelForCausalLM.from_pretrained(name, cache_dir='./model/'
                            , use_auth_token=auth_token, torch_dtype=torch.float16, 
                            rope_scaling={"type": "dynamic", "factor": 2}, load_in_8bit=True) 

    return tokenizer, model
tokenizer, model = get_tokenizer_model()



/home/chats/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/chats/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/chats/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/home/chats/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


RuntimeError: No GPU found. A GPU is needed for quantization.

In [7]:

# Create a system prompt 
system_prompt = """<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as 
helpfully as possible, while being safe. Your answers should not include
any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain 
why instead of answering something not correct. If you don't know the answer 
to a question, please don't share false information.

Your goal is to provide answers relating to the financial performance of 
the company.<</SYS>>
"""
# Throw together the query wrapper
query_wrapper_prompt = SimpleInputPrompt("{query_str} [/INST]")

# Create a HF LLM using the llama index wrapper 
llm = HuggingFaceLLM(context_window=4096,
                    max_new_tokens=256,
                    system_prompt=system_prompt,
                    query_wrapper_prompt=query_wrapper_prompt,
                    model=model,
                    tokenizer=tokenizer)

# Create and dl embeddings instance  
embeddings=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
)

# Create new service context instance
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embeddings
)
# And set the service context
set_global_service_context(service_context)

# Download PDF Loader 
PyMuPDFReader = download_loader("PyMuPDFReader")
# Create PDF Loader
loader = PyMuPDFReader()
# Load documents 
documents = SimpleDirectoryReader('/home/chats/data/YuluGyan').load_data()
#documents = loader.load(file_path=Path('./data/annualreport.pdf'), metadata=True)

# Create an index - we'll be able to query this in a sec
index = VectorStoreIndex.from_documents(documents)
# Setup index query engine using LLM 
query_engine = index.as_query_engine()

# Create centered main title 

In [9]:
response = query_engine.query("What all products yullu have?")
response.response

' Based on the provided context information, Yulu offers the following products:\n1. Yulu Miracle: A shared electric 2-wheeler designed for first-and-last mile connectivity, intra-city mobility, and commute options.\n2. Yulu DeX: A purpose-built delivery vehicle designed for last-mile delivery companies and delivery partners.\n3. Yulu Wynn: A revolutionary electric 2-wheeler designed for easy mobility for urban families, with industry-first features like truly-keyless access and instant family-sharing.\n4. A complete suite of mobility solutions, including purpose-built, green, and efficient EVs, an intelligent IoT platform, a fleet management system powered by AI/ML, and an intuitive customer app.\nYulu also offers a shared, smart, and sustainable Public Bike Sharing (PBS) system with the objective of reducing congestion, pollution, and enhancing economic productivity. They provide seamless connectivity with public transport hubs, such as Railway Stations, Bus Rapid Transit Systems, Me

In [ ]:
HUGGINGFACE_TOKEN = ""

MODEL = "meta-llama/Llama-2-7b-chat-hf"

In [13]:
response = query_engine.query("who are founders of yulu?")
response.get_formatted_sources()

'> Source (Doc id: 5d45f794-cb44-4f48-993f-811ce8c3107b): Gupta, Yulu provides urban Mobility-as-a-Service (MaaS) in Bengaluru, Mumbai and Delhi-NCR and is...\n\n> Source (Doc id: 797d1b9f-ad45-4ff8-b447-1f5fa058f3e5): equity analysis & actionable investment ideasSubscribeCurated markets data, exclusive trading rec...'

In [12]:
response

Response(response=" Thank you for reaching out! I'm happy to help you with your query.\nYulu's mission is to provide a convenient, affordable, and sustainable transportation solution for short distances. They aim to reduce traffic congestion and pollution in urban areas by promoting the use of electric bicycles and scooters.\nAs for the cost of a ride on Yulu, the prices vary depending on the distance traveled and the type of vehicle used. Here are the current prices for Yulu's services:\n* Yulu Bike: ₹10 per km (minimum fare ₹100)\n* Yulu Scoot: ₹15 per km (minimum fare ₹150)\nPlease note that these prices are subject to change, and you may want to check the Yulu website or app for the most up-to-date pricing information.\nI hope this helps! Let me know if you have any other questions.", source_nodes=[NodeWithScore(node=TextNode(id_='360d382a-c32a-42e6-8303-2a21377212a1', embedding=None, metadata={'page_label': '20', 'file_name': 'Yulu_website_data.pdf'}, excluded_embed_metadata_keys=

In [1]:
!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin


--2023-10-15 20:44:26--  https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin
Resolving gpt4all.io (gpt4all.io)... 172.67.71.169, 104.26.1.159, 104.26.0.159, ...
Connecting to gpt4all.io (gpt4all.io)|172.67.71.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785248281 (3.5G)
Saving to: ‘ggml-gpt4all-j-v1.3-groovy.bin’

ggml-gpt4all-j-v1.3 100%[===================>]   3.52G  39.1MB/s    in 88s     

2023-10-15 20:45:55 (40.8 MB/s) - ‘ggml-gpt4all-j-v1.3-groovy.bin’ saved [3785248281/3785248281]



In [5]:
import gpt4all


In [1]:
from gpt4all import GPT4All
model = GPT4All("orca-mini-3b.ggmlv3.q4_0.bin")
output = model.generate("The capital of France is ")
print(output)

100%|██████████████████████████████████████| 1.93G/1.93G [00:07<00:00, 252MiB/s]


Model downloaded at:  /home/chats/.cache/gpt4all/orca-mini-3b.ggmlv3.q4_0.bin


llama.cpp: loading model from /home/chats/.cache/gpt4all/orca-mini-3b.ggmlv3.q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 3200
llama_model_load_internal: n_mult     = 240
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 26
llama_model_load_internal: n_rot      = 100
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 8640
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 3B
llama_model_load_internal: ggml ctx size =    0.06 MB
llama_model_load_internal: mem required  = 2862.72 MB (+  682.00 MB per state)
.
llama_init_from_file: kv self size  =  650.00 MB


TypeError: LLModel.prompt_model() got an unexpected keyword argument 'max_tokens'

In [2]:
output = model.generate("The capital of France is ")
print(output)

1. Paris, which is also the capital of France, is located in the northern part of the country. Paris is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral and

Exception ignored on calling ctypes callback function: <function LLModel._response_callback at 0x7fe9bb385360>
Traceback (most recent call last):
  File "/home/chats/.local/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 328, in _response_callback
    @staticmethod
KeyboardInterrupt: 


 Louvre Museum.
Paris is also known for its fashion, cuisine and art. It is a popular tourist destination, attracting millions of visitors every year.
2. LondonThe capital of the United Kingdom, London is located in the southern part of the country. London is known for its iconic landmarks such as the Big Ben, Buckingham Palace and the Tower of London.
London is also known for

Exception ignored on calling ctypes callback function: <function LLModel._response_callback at 0x7fe9bb385360>
Traceback (most recent call last):
  File "/home/chats/.local/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 328, in _response_callback
    @staticmethod
KeyboardInterrupt: 


 history
1. Paris, which is also the capital of France, is located in the northern part of the country. Paris is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral and Louvre Museum.
Paris is also known for its fashion, cuisine and art. It is a popular tourist destination, attracting millions of visitors every year.
2. LondonThe capital of the United Kingdom, London is located in the southern part of the country. London is known for its iconic landmarks such as the Big Ben, Buckingham Palace and the Tower of London.
London is also known for history


In [ ]:
model.generate